In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
def Normalization(x):
    max_val = max(x)
    min_val = min(x)
    return [(i-min_val)/(max_val-min_val) for i in x]

In [3]:
def weight_lost_cal(real, pre):
    
    test_auc = metrics.roc_auc_score(real, pre)
    fpr, tpr, thresholds = metrics.roc_curve(real, pre, pos_label=1) 
    score=0.4*tpr[np.where(fpr>=0.001)[0][0]]+0.3*tpr[np.where(fpr>=0.005)[0][0]]+0.3*tpr[np.where(fpr>=0.01)[0][0]] 
    
    print("AUC value = " + str(test_auc))
    print("Score = " + str(score))
    return score

In [4]:
# read data
root = "./data/"
data_train = pd.read_csv(root + "atec_anti_fraud_train.csv")
# data_train_add = pd.read_csv(root + "atec_anti_fraud_train_added.csv")
data_test = pd.read_csv(root + "atec_anti_fraud_test_a.csv")

In [5]:
# data preprocessing
# 是否要采样
# data_train = data_train.sample(100000)
# data_train = data_train[data_train['label'] != -1]

data_test = data_test.drop(['id'], 1)
data_test = data_test.drop(['date'], 1)

data_train = data_train.drop(['id'], 1)
data_train = data_train.drop(['date'], 1)
# data_train = pd.concat([data_train,data_train_add], sort=True)

data_train['label'] = np.abs(data_train['label'])
train_y = data_train['label']
data_train = data_train.drop(['label'], 1)

# 删除特征
b = ['20','22','24','26','28','30','32','34','46','47','48','50','52','53']        
c = [str(i) for i in range(64,72)]
d = [str(i) for i in range(111,155)]

drop_col = ['f'+i for i in (b+c+d)]

data_test = data_test.drop(drop_col,1)
data_train = data_train.drop(drop_col,1)

In [6]:
train_x = data_train
test_x = data_test
# train_y = np.array(train_y).reshape(-1, 1)

# train_x = np.array(train_x)
# train_y = np.array(train_y).reshape(-1, 1)
# test_x = np.array(test_x)

X = np.array(train_x)
Y = np.array(train_y)
X_test = test_x

In [7]:
X_test = test_x

In [ ]:
# train
# 交叉验证取平均
k = 5
kf = KFold(n_splits = k, shuffle=True, random_state=2)
# kf = KFold(n_splits=k)

eta = 0.05
result = 0
xgb_result = 0
importances = 0
for train_index, test_index in kf.split(X):
    X1_train, X1_test = X[train_index], X[test_index]
    Y1_train, Y1_test = Y[train_index], Y[test_index]

#     data_train = lgb.Dataset(X1_train, Y1_train)
#     data_vtest = lgb.Dataset(X1_test, Y1_test)
#     data_rtest = lgb.Dataset(X_test)
    print("Initializing......")
    gbdt = GradientBoostingClassifier(n_estimators=10, max_depth=6)
    
    print("Training......")
    bst = gbdt.fit(X1_train, Y1_train)
    
    print("TEST......")
    
    y_predprob = gbdt.predict_proba(X1_test)[:,1]
    print("AUC: %f" % metrics.roc_auc_score(Y1_test, y_predprob))
    
    result = weight_lost_cal(Y1_test.reshape(-1, 1), y_predprob.reshape(-1, 1))
    
    xgb_result += gbdt.predict(X_test)  # 测试集   

    print("done...")
    
# 计算结果
result /= k
xgb_result /= k


In [10]:
print(result)
print(max(xgb_result))
print(min(xgb_result))

0.45580126854216685
0.8741891690562273
0.00018952433537072687


In [11]:
# 输出
data_test = pd.read_csv(root + "atec_anti_fraud_test_a.csv")
data_test['score'] = xgb_result
data_test['score'].round(decimals=5)
data = data_test[['id', 'score']]  #选择表格中的两列
data.to_csv(root + "result_717_1000_V2.csv", index=None, encoding='utf8')